# Darcy equation

In this tutorial we present how to solve a Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon) in themoving domain case (the upper boundary will move).  The unkwons are the velocity $u$, the elevation head $h$ and the height of the upper boundary $\eta$.

Let $\Omega=(0,1)\times(0,\eta)$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$K$ the matrix permeability, we want to solve the following problem: find $(\bm{u}, h)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
K^{-1} {\bm{u}} + \nabla h = {0}\\
S_s \frac{\partial{h}}{\partial t} + \nabla \cdot {u} = f
\end{array}
&\text{in } \Omega
\end{array}
\right.
$$

In order to solve the problem, we will perfom a change of coordinates to a reference domain $\hat{\Omega}=(0,1)^2$ through the (linear) trasnformation $R : \Omega \rightarrow \hat{\Omega}$ (and its inverse function $D : \hat{\Omega} \rightarrow \Omega$).
Recall that $\hat{\nabla}R=(\nabla D)^{-1}$.

Let $\hat{h}$ and $\hat{\bm{u}}$ be $h$ and $\bm{u}$ respectevely in the reference domain and let $\hat{K}$ be the transformed permeability matrix, defined as $\hat{K}=det(\hat{\nabla}D) (\hat{\nabla} D)^{-1} K (\hat{\nabla} D)^{-T}$.

The equation describing the motion of $\partial_{top}\Omega$ is:
$$

\phi \frac{\partial \eta}{\partial t} = \hat{u_3}

$$

The transformed equations in $\hat{\Omega}$ is:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\hat{K}(\hat{h})^{-1} {\hat{u}} + \hat{\nabla} \hat{h} = {0}\\
\hat{S}_s \frac{\partial{\hat{h}}}{\partial t} + \hat{\nabla} \cdot {\hat{\bm{u}}} = f
\end{array}
&\text{in } \hat{\Omega}
\end{array}
\right.
$$
with boundary conditions:
$$ \hat{h} = \eta \text{ on } \Gamma \qquad \hat{h} = \ell \text{ on } \Gamma_D \qquad \hat{\bm{\nu}} \cdot \hat{\bm{u}} = 0 \text{ on } \Gamma_N$$

The weak formulation will be:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\int_{\Omega}\hat{K}(\hat{h})^{-1} {\bm{\hat{u}}} \cdot \bm{v} \, d\Omega - \int_{\Omega} h \hat{\nabla} \cdot {\hat{\bm{v}}} \, d\Omega = - \int_{\Gamma_D} h \bm{v} \cdot \bm{\nu} \, d\Omega - \int_{\Gamma} \eta \bm{v} \cdot \bm{\nu} \, d\Omega\\
\int_{\Omega} \hat{S}_s \frac{\partial{\hat{h}}}{\partial t} v \, d\Omega + \int_{\Omega} \hat{\nabla} \cdot {\hat{\bm{u}}} v \, d\Omega = \int_{\Omega} fv \, d\Omega\\
\int_{\Gamma} \phi \frac{\partial \eta}{\partial t} v \, d\sigma = \int_{\Gamma} \hat{u_3} v \, d\sigma
\end{array}
\end{array}
\right.
$$

For the time discretization, we will employ a backward Euler scheme:

$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\int_{\Omega}\hat{K}(\hat{h}^{n+1})^{-1} {\bm{\hat{u}}^{n+1}} \cdot \bm{v} \, d\Omega - \int_{\Omega} h^{n+1} \hat{\nabla} \cdot {\hat{\bm{v}}} \, d\Omega = - \int_{\Gamma_D} h^{n+1} \bm{v} \cdot \bm{\nu} \, d\Omega - \int_{\Gamma} \eta^{n+1} \bm{v} \cdot \bm{\nu} \, d\Omega\\
\int_{\Omega} \hat{S}_s^{n+1} \frac{\hat{h}^{n+1} - \hat{h}^{n}}{\Delta t} v \, d\Omega + \int_{\Omega} \hat{\nabla} \cdot {\hat{\bm{u}}^{n+1}} v \, d\Omega = \int_{\Omega} f^{n+1}v \, d\Omega\\
\int_{\Gamma} \phi \eta^{n+1} v \, d\sigma = \Delta t \int_{\Gamma} \hat{\bm{u}}^{n+1} \cdot \bm{\nu} v \, d\sigma + \int_{\Gamma} \phi \eta^{n} v \, d\sigma
\end{array}
\end{array}
\right.
$$

To deal with the non-linear term, we will employ a simple Picard scheme:

$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\int_{\Omega}\hat{K}(\hat{h}^{n+1}_k)^{-1} {\bm{\hat{u}_{k+1}^{n+1}}} \cdot \bm{v} \, d\Omega - \int_{\Omega} h^{n+1}_{k+1} \hat{\nabla} \cdot {\hat{\bm{v}}} \, d\Omega + \int_{\Gamma} \eta^{n+1}_{k+1} \bm{v} \cdot \bm{\nu} \, d\Omega= - \int_{\Gamma_D} h^{n+1} \bm{v} \cdot \bm{\nu} \, d\Omega\\
\Delta t \int_{\Omega} \hat{\nabla} \cdot {\hat{\bm{u}}^{n+1}_{k+1}} v \, d\Omega + \int_{\Omega} \hat{S}_s \hat{h}^{n+1}_{k+1} v \, d\Omega = \int_{\Omega} \hat{S}_s \hat{h}^{n} v \, d\Omega + \Delta t \int_{\Omega} f^{n+1}v \, d\Omega\\
- \Delta t \int_{\Gamma} \hat{\bm{u}}^{n+1}_{k+1} \cdot \bm{\nu} v \, d\sigma + \int_{\Gamma} \phi \eta^{n+1}_{k+1} v \, d\sigma = \int_{\Gamma} \phi \eta^{n} v \, d\sigma
\end{array}
\end{array}
\right.
$$

The matrix formulation will be:

$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
M_u(\bm{h}^{n+1}_{k+1}) \bm{u}^{n+1}_{k+1} + B^T\bm{h}^{n+1}_{k+1} + B_{\Gamma} \bm{\eta}^{n+1}_{k+1}= \bm{BC}^{n+1}\\
- \Delta t B \hat{\bm{u}}^{n+1}_{k+1} + S_s M_{h} \bm{\hat{h}^{n+1}_{k+1}} = \Delta t \bm{F}^{n+1} + S_s M_{h} \bm{\hat{h}^{n}}\\
- \Delta t B_{\Gamma}^T \hat{\bm{u}}^{n+1}_{k+1} + \phi M_{\Gamma} \bm{\eta^{n+1}_{k+1}} = \phi M_{\Gamma} \bm{\eta^{n}}
\end{array}
\end{array}
\right.
$$

$$
\left(
\begin{array}{cc} 
M_u(\bm{h^{n+1}_k}) & B^T & B_{\Gamma}\\
-\Delta t B & S_s M_h & 0\\
-\Delta t B^T_{\Gamma} & 0 & \phi M_{\Gamma}
\end{array}
\right)
\left(
\begin{array}{c} 
\bm{u^{n+1}_{k+1}}\\ 
\bm{h^{n+1}_{k+1}}\\
\bm{\eta^{n+1}_{k+1}}
\end{array}
\right)
=\left(
\begin{array}{c} 
\bm{BC}^{n+1}\\ 
\Delta t \bm{F}^{n+1} + S_s M_h \bm{h}^n\\
\phi M_{\Gamma} \bm{\eta}^n
\end{array}
\right)
$$

We will start to test the method in the case $M_u(\bm{h_k}^{n+1})=\bm{I}$

In [1]:
import numpy as np
import scipy.sparse as sps
import os
import shutil

import porepy as pp
import pygeon as pg

In [2]:
output_directory = 'output'

We create now the grid, since we will use a Raviart-Thomas approximation for ${q}$ we are restricted to simplices. In this example we consider a 2-dimensional structured grid, but the presented code will work also in 1d and 3d. PyGeoN works with mixed-dimensional grids, so we need to convert the grid.

In [3]:
N = 10
sd = pp.StructuredTriangleGrid([N] * 2, [1] * 2)
# convert the grid into a mixed-dimensional grid
mdg = pp.meshing.subdomains_to_mdg([sd])

dt = 0.1
T = 10
S_s = 0.3
phi = 0.1

With the following code we set the data, in particular the permeability tensor and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [4]:
key = "flow"
bc_val = []
bc_ess = []
initial_pressure = []

velocity_discretization_field = pg.RT0(key)
head_discretization_field = pg.PwConstants(key)

sd, data = mdg.subdomains(return_data=True)[0]

# permeability tensor
perm = pp.SecondOrderTensor(np.ones(sd.num_cells))
parameters = {
    "second_order_tensor": perm,
}

pp.initialize_data(sd, data, key, parameters)
    
# with the following steps we identify the portions of the boundary
# to impose the boundary conditions
left_right = np.logical_or(sd.face_centers[0, :] == 0,  sd.face_centers[0, :] == 1)

bottom = sd.face_centers[1, :] == 0
top    = sd.face_centers[1, :] == 1

ess_p_dofs = np.zeros(head_discretization_field.ndof(sd), dtype=bool)

def h_bc(x): return 1
def initial_h_func(x): return 1-x[1]

bc_val.append(-velocity_discretization_field.assemble_nat_bc(sd, h_bc, bottom))

bc_ess.append(np.hstack((left_right, ess_p_dofs, np.zeros(N, dtype=bool))))

In [5]:
def assemble_gamma_matrices():
    B_rows = []

    faces = np.where( top == True )[0]

    for i in range(N):
        tmp_vect = np.zeros_like(top, dtype=bool)
        tmp_vect[ faces[i] ] = True
        
        B_rows.append( velocity_discretization_field.assemble_nat_bc(sd, h_bc, tmp_vect).T )
    
    return np.vstack(B_rows), np.eye(N=N)

In [6]:
B_gamma, M_gamma = assemble_gamma_matrices()

$$
\left(
\begin{array}{cc} 
M_u(\bm{h^{n+1}_k}) & B^T & B_{\Gamma}\\
-\Delta t B & S_s M_h & 0\\
-\Delta t B^T_{\Gamma} & 0 & \phi M_{\Gamma}
\end{array}
\right)
\left(
\begin{array}{c} 
\bm{u^{n+1}_{k+1}}\\ 
\bm{h^{n+1}_{k+1}}\\
\bm{\eta^{n+1}_{k+1}}
\end{array}
\right)
=\left(
\begin{array}{c} 
\bm{BC}^{n+1}\\ 
\Delta t \bm{F}^{n+1} + S_s M_h \bm{h}^n\\
\phi M_{\Gamma} \bm{\eta}^n
\end{array}
\right)
$$

In [7]:
# construct the local matrices
mass = pg.face_mass(mdg)
div = - pg.cell_mass(mdg, head_discretization_field) @ pg.div(mdg)

# get the degrees of freedom for each variable
dof_p, dof_q = div.shape
dof_eta = B_gamma.shape[0]

mass.shape, div.shape, B_gamma.shape

((320, 320), (200, 320), (10, 320))

In [8]:
# assemble the right-hand side
fixex_rhs = np.zeros(dof_p + dof_q + dof_eta)
fixex_rhs[:dof_q] += np.hstack(bc_val)

In [9]:
# assemble the saddle point problem
spp = sps.bmat([[    mass,                                                         div.T,          B_gamma.T], 
                [    -div, S_s / dt * head_discretization_field.assemble_mass_matrix(sd),               None],
                [-B_gamma,                                                          None, phi / dt * M_gamma]], format="csc")

In [10]:
def save_step(sol, proj_q, proj_psi, saver, i):
    ins = list()

    ins.append((sd, "cell_q", ( proj_q @ sol[-1][:dof_q] ).reshape((3, -1), order="F")))
    ins.append((sd, "cell_p", proj_psi @ sol[-1][dof_q:(dof_q+dof_p)]))
    print( sol[-1][-dof_eta:] )
    
    saver.write_vtu(ins, time_step=i)

In [11]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [12]:
sol = [np.zeros(dof_p + dof_q + dof_eta)]
sol[-1][dof_q:(dof_q+dof_p)] = head_discretization_field.interpolate(sd, initial_h_func)
sol[-1][-dof_eta:] = np.ones_like(sol[-1][-dof_eta:])

proj_q = velocity_discretization_field.eval_at_cell_centers(sd)
proj_psi = head_discretization_field.eval_at_cell_centers(sd)

saver = pp.Exporter(mdg, 'sol', folder_name=output_directory)
save_step(sol, proj_q, proj_psi, saver, 0)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [13]:
for i in range(1, int(T/dt)+1):
    rhs = fixex_rhs.copy()
    rhs[dof_q:(dof_q+dof_p)] += S_s / dt * head_discretization_field.assemble_mass_matrix(sd) @ sol[-1][dof_q:(dof_q+dof_p)]
    rhs[-dof_eta:] += phi / dt * M_gamma @ sol[-1][-dof_eta:]
    
    ls = pg.LinearSystem(spp, rhs)
    ls.flag_ess_bc(np.hstack(bc_ess), np.zeros(dof_q + dof_p + dof_eta))
    sol.append( ls.solve() )

    save_step(sol, proj_q, proj_psi, saver, i)

    print('Time ' + str(dt * i) )

saver.write_pvd([t * dt for t in range(int(T/dt)+1)])

[0.92892218 0.9289664  0.92899216 0.92900934 0.9290226  0.92903472
 0.92904798 0.92906517 0.92909094 0.92913515]
Time 0.1
[0.92525528 0.9252677  0.92528161 0.92529455 0.92530654 0.92531816
 0.92533014 0.92534308 0.92535697 0.92536939]
Time 0.2
[0.92954913 0.92955468 0.92956348 0.92957309 0.92958276 0.9295924
 0.92960207 0.92961168 0.92962048 0.92962604]
Time 0.30000000000000004
[0.93499648 0.93499998 0.93500617 0.93501348 0.93502119 0.935029
 0.93503671 0.93504401 0.9350502  0.93505371]
Time 0.4
[0.94031638 0.94031897 0.94032355 0.94032919 0.94033532 0.94034159
 0.94034772 0.94035336 0.94035794 0.94036053]
Time 0.5
[0.94526504 0.94526705 0.94527054 0.94527494 0.94527981 0.94528481
 0.94528968 0.94529408 0.94529757 0.94529958]
Time 0.6000000000000001
[0.94981733 0.94981893 0.94982164 0.94982509 0.94982895 0.94983294
 0.9498368  0.94984025 0.94984296 0.94984456]
Time 0.7000000000000001
[0.95399389 0.95399518 0.9539973  0.95400002 0.95400309 0.95400627
 0.95400933 0.95401205 0.95401418 0.

Time 6.2
[0.99961321 0.99961322 0.99961322 0.99961322 0.99961323 0.99961323
 0.99961323 0.99961324 0.99961324 0.99961325]
Time 6.300000000000001
[0.9996454  0.9996454  0.9996454  0.99964541 0.99964541 0.99964542
 0.99964542 0.99964542 0.99964543 0.99964543]
Time 6.4
[0.99967491 0.99967491 0.99967491 0.99967492 0.99967492 0.99967492
 0.99967492 0.99967493 0.99967493 0.99967493]
Time 6.5
[0.99970196 0.99970196 0.99970196 0.99970197 0.99970197 0.99970197
 0.99970198 0.99970198 0.99970198 0.99970198]
Time 6.6000000000000005
[0.99972676 0.99972676 0.99972676 0.99972677 0.99972677 0.99972677
 0.99972678 0.99972678 0.99972678 0.99972678]
Time 6.7
[0.9997495  0.9997495  0.9997495  0.9997495  0.99974951 0.99974951
 0.99974951 0.99974951 0.99974952 0.99974952]
Time 6.800000000000001
[0.99977034 0.99977034 0.99977035 0.99977035 0.99977035 0.99977035
 0.99977036 0.99977036 0.99977036 0.99977036]
Time 6.9
[0.99978945 0.99978946 0.99978946 0.99978946 0.99978946 0.99978946
 0.99978947 0.99978947 0.99